# Notebook export

A couple of rules:
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable in isolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```

 - eliminate circular dependencies the same way unintended recursion is avoided, create a condition:
 
ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
var funcA = require('ModuleA.js');
```

Becomes:

ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
if(typeof funcA === 'undefined') {
    var funcA = require('ModuleA.js');
}
```

 - parameters are listed in most specific left to least specific right, i.e. function(filter, context) would mean filter is only used for this function, whereas context may be passed in to this function as well as other functions.  "filter" is on the left because it is specifically used just for this function, context is on the right because it might contain a path on the filesystem, or some options.
 - Entry cells, i.e. cells that are intended to be called by a service, coordinate interactions between multiple services - should not contain catch blocks so that the task scheduler will fail and log the last error.


readme.md?

In [ ]:
// readme placeholder

get cell extension?


In [ ]:
// TODO: replace with a library?

// TODO: move into interpret notebook cell?

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)
            || (cell.questions[0] || '').includes('test')) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: '
            + (cell.filename || notebook)
            + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


get imports recursively?

TODO: use this in file-system graph

TODO: AST compatible map of import functions across all languages
Python `__import__`, Java: `import`, C#: `using`, C++: `#import`.  Basic functionality of JetBrains intellisense.

TODO: import gist as notebook

TODO: import random instructions as notebook using `npx` remote code execution



In [ ]:
var importer = require('../Core');
var getExports = importer.import('get exports from source');
var getImports = importer.import('get imports from source');
var niceName = importer.import('rename cell nice name');
var addAst = importer.import('get all ast properties?')

// TODO: remove this using a recursion pattern
function getImportsRecursively(allImports, searches) {
    var imports = {};
    searches = typeof searches === 'string' ? [searches] : searches;
    var results = importer.interpret(searches);
    // normalize input incase importing an array like .import(['search'])
    searches.forEach((search, i) => {
        // skip if already included
        if(typeof allImports[search] !== 'undefined') return;
        allImports[search] = results[i];
        var result = typeof results[i][0] === 'undefined' ? [results[i]] : results[i];
        result.forEach(cell => {
            addAst(cell);
            cell.name = niceName(cell);
            allImports[cell.id] = cell;
            imports[cell.id] = cell.imports || [];
            imports[cell.id].forEach(arr => {
                Object.assign(imports, getImportsRecursively(allImports, arr));
            });
        })
    });
    return imports;
}

module.exports = getImportsRecursively;



rename cell to a nice name?

TODO: move this to Core notebook?

In [ ]:
var importer = require('../Core');
var getExtension = importer.import('cell extension')

// get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    var name = cell.questions[0].replace('?', '')
        .replace(/[^a-z0-9]+|\btest\b/igm, ' ')
        .trim()
        .replace(/\s+/igm, '-')
        + getExtension(cell);
    if(name.toLowerCase().includes('readme')) {
        name = 'readme.md';
    }
    if(name.toLowerCase().includes('package-json')) {
        name = 'package.json';
    }
    return name;
}

module.exports = niceName;


include require like import?

The import function essentially tries to return a function if that is all the module exports.  This is to prevent those annoying upgrades like NPM has where you have to extrapolate the import like ```let {export} = require('module')``` versus ```let export = require('module')```.

This method is used in Core, and this garuntees the same thing happens when including the file directly.

The multiple include is used when importing an entire notebook worth of functions. It always returns the indexed object, but also tries to assign the function names to the object so they can be extralopolated easily.



In [ ]:
var requireTemplate = path => `require('./${path}')`;

var importTemplate = (fnName, path) => `
var ${fnName} = (${includeTemplate.toString()})(${requireTemplate(path)});`

var includeTemplate = (r) => typeof r === 'function'
    ? r
    : (r && Object.keys(r).length === 1
       && typeof r[Object.keys(r)[0]] === 'function'
        ? r[Object.keys(r)[0]]
        : r);

var multipleImportTemplate = (imports) => `[${imports.map(i => requireTemplate(i)).join(', ')}]
.reduce((acc, req, i) => {
    var r = (${includeTemplate.toString()})(req);
    if(typeof r === 'function') {
        acc[r.name] = r;
    }
    acc[i] = r;
    return acc;
}, {})`

var importsTemplate = addImports => `({${addImports.reduce((arr, id) => arr.concat([`
    ${JSON.stringify(id)}: (ctx) => importNotebook(${JSON.stringify(id)})`]), [])
    .join(`,\n        `)}})[r.id] || true`;

var cachedTemplate = cells => `((cachedCells) => Object.keys(cachedCells)
    .map(n => cacheCells(cachedCells[n], n)))
(${JSON.stringify(cells, null, 4)})`

module.exports = {
    importTemplate,
    multipleImportTemplate,
    requireTemplate,
    importsTemplate,
    cachedTemplate
}

replace notebook imports?

automatic jupyter dependency injector?


In [ ]:
var escodegen = require('escodegen');
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {requireTemplate, importTemplate, multipleImportTemplate} = importer.import('include require like import');
var {GET_CALLED, GET_DECLARED, GET_PARENT, GET_CORE,
     GET_CORENAME, GET_CALLER, GET_IMPORT} = importer.import('common ast queries');
var injectImports = importer.import('inject cells notebooks webpack');
var getImports = importer.import('get imports from source');

var insertMissing = (ast, fn, name) => 
    ast.body.unshift.apply(ast.body, ...getArrayAST('/*', importTemplate(fn, name)))

// TODO: use this in require('../Core'), automatically import into global || window
function addImports(ast, allImports) {
    Object.values(allImports)
        .forEach(cell => (cell.exports || [])
            // returns true if there is a call to a function that is not defined in the code cell 
            .forEach(e => {
                if(0 < getArrayAST(GET_CALLED(e), ast).length
                    && 0 === getArrayAST(GET_DECLARED(e), ast).length) {
                    insertMissing(ast, e, cell.name)
                }
            }))
}

function replaceCore(ast) {
    var impParent = getArrayAST(GET_PARENT(GET_CORE), ast)[0];
    if(impParent) {
        var impDeclare = getArrayAST(GET_CORE, impParent)[0];
        var impName = getArrayAST(GET_CORENAME, impDeclare)[0].name;
        impParent.splice(impParent.indexOf(impDeclare), 1);
        var impUsage = getArrayAST(GET_CALLER(impName), ast);
        impUsage.forEach(parent => {
            parent.callee = parent.callee.property;
            if(parent.callee.name === 'import') parent.callee.name = 'importNotebook'
        });
    }
}

function replaceImport(ast, i, allImports) {
    let result = typeof i === 'string' ? allImports[i] : i.map(i => allImports[i]);
    if (typeof result === 'undefined' || result.length === 0) {
        throw new Error(`Replacement not found "${i}"`);
    }
    // find the import matching the string i
    const importStr = typeof i === 'string' ? i : i[0];
    var impCall = getArrayAST(GET_IMPORT(importStr), ast)[0];
    if(!impCall) {
        if(!i.includes('import.ipynb')) {
            throw new Error(`Replacement not found "${i}"`);
        }
        return;
    }
    
    var template;
    if(result.length > 1 || (i.includes('.ipynb') && !i.includes('['))) {
        const imports = result.map(r => r.name);
        template = multipleImportTemplate(imports);
    } else {
        result = result[0] || result;
        template = requireTemplate(result.name);
    }
    Object.assign(impCall, getArrayAST('//CallExpression', template)[0]);
}

function replaceImports(cell, allImports) {
    var ast = getArrayAST('*', cell.code)[0];
    if(cell.notebook === 'import.ipynb' && cell.exports[0] === 'interpret') {
        injectImports(ast, allImports);
    }
    replaceCore(ast);
    getImports(ast).forEach(i => replaceImport(ast, i, allImports));
    addImports(ast, allImports);
    
    escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace);
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    });
}

// TODO: replace getEnvironment or process.env.var calls with actual env known ahead of time
// TODO: replace "get rpc spec from notebooks" with static result

module.exports = replaceImports;


export deploy notebook coordinator?

TODO: automatically find missing imports from distrib/Github/3rd party sources in any language?



In [ ]:
var fs = require('fs');
var path = require('path');
var minimatch = require('minimatch');
var mkdirpSync = importer.import('mkdirp');

var importer = require('../Core');
var authorTemplate = importer.import('authoring header template');
var replaceImports = importer.import('replace notebook imports');
var getImportsRecursively = importer.import('get imports recursively');
var projectRelatives = importer.import('fix project paths');
var delintCode = importer.import('delint notebooks');

// searches are the top level cells starting the import tree
function exportAndDeploy(searches, projectOutput, matchOutput) {
    projectOutput = projectOutput || process.env.EXPORT_PATH || path.join(path.resolve(__dirname), '../.output');
    
    const allImports = {};
    const imports = getImportsRecursively(allImports, searches);
    
    var core = [];
    Object.keys(imports)
        .filter(i => allImports[i].questions.length > 0
               || allImports[i].questions[0].trim() != '')
        .forEach((e, i) => {
            var exportedCode;
            assert(!allImports[e].name.match(/^\./), `No filename ${e} ${allImports[e].questions}!`);
            // some special exceptions with file-naming
            if(allImports[e].name === 'readme.md') {
                exportedCode = allImports[e].markdown;
            } else if (allImports[e].language === 'javascript' && allImports[e].ast) {
                exportedCode = replaceImports(allImports[e], allImports);
                const delinted = delintCode(exportedCode)[0];
                exportedCode = delinted.fixed || delinted.source;
            } else {
                exportedCode = allImports[e].code;
            }
            
            outputExport(exportedCode, allImports[e], projectOutput, matchOutput || {})
        });
    
    return projectRelatives(projectOutput)
    // TODO: output packed cells and cache
    // TODO: zip and upload to AWS
}

var getDirectory = (match) => match[match.length-1] === '/'
    ? match
    : (fs.existsSync(match) && fs.lstatSync(match).isDirectory()
       ? match
       : path.dirname(match));


// TODO: move this to filesystem utility?
var matchFilename = (filename, matchOutput, projectOutput) => Object.keys(matchOutput)
    .filter(match => minimatch(filename, match))
    .map(match => {
        var projectMatch = path.join(projectOutput, matchOutput[match])
        var dir = getDirectory(projectMatch);
        return path.join(dir, path.basename(projectMatch) === path.basename(dir)
            ? filename
            : path.basename(matchOutput[match]));
    });

function outputExport(exportedCode, cell, projectOutput, matchOutput) {
    // emit the file in every location request
    matchFilename(cell.name, matchOutput, projectOutput).forEach(filename => {
        // create directory if needed
        mkdirpSync(path.dirname(filename));
        console.log(`emitting ${filename}`);
        // add a code block header
        if(path.extname(filename) == '.js' || path.extname(filename) == '.cs') {
            exportedCode = authorTemplate(cell.markdown, exportedCode)
        }
        fs.writeFileSync(filename, exportedCode);
    })
}

module.exports = exportAndDeploy;


authoring header template?


In [ ]:

// TODO: move this heading authoring template to utility function
function authorTemplate(markdown, code) {
    return `
/**
 * Written by Brian Cullinan, exported using magic.
 * Copyright (c) ${(new Date()).getFullYear()} by Brian Cullinan, All rights reserved.
 *
${((markdown || '') + '').split('\n').map(l => ' * ' + l).join('\n')}
 *
 **/

${code}`;
}

module.exports = authorTemplate;

inject cells in to notebooks when built with webpack?


In [ ]:
var path = require('path');
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {importsTemplate, cachedTemplate} = importer.import('include require like import');

function cellsToNotebook(allImports) {
    return Object.keys(allImports)
        .map(k => allImports[k].filename)
        .filter((n, i, arr) => arr.indexOf(n) === i)
        .reduce((acc, k) => {
            acc[k] = importer.interpret(path.basename(k)).reduce((acc, cell) => {
                acc[cell.to] = {
                    source: cell.source,
                    language: cell.language,
                    cell_type: 'code'
                }
                cell.markdown.forEach((m, i) => {
                    acc[cell.from + i] = {
                        source: m.split(/\n/ig).map(l => l + '\n'),
                        language: cell.language,
                        cell_type: 'markdown'
                    };
                });
                return acc;
            }, [])
            return acc;
        }, {});
}

// TODO: this should be a test in the Core notebook
// inject cellCache and cellIds loaded in to the interpreter so notebooks don't need to be uploaded,
//   and webpack can tree-shake on functions already packed
function injectImports(ast, allImports) {
    var cacheBody = getArrayAST(`//*[/*/ExpressionStatement/CallExpression[/Identifier[@name == 'cacheAll']]]`, ast)[0];
    var firstCache = getArrayAST(`//ExpressionStatement[/CallExpression[/Identifier[@name == 'cacheAll']]]`, cacheBody)[0];
    var cacheCode = getArrayAST('*', cachedTemplate(cellsToNotebook(allImports)))[0];
    cacheBody.body.splice(cacheBody.body.indexOf(firstCache), 1, ...cacheCode.body);
    
    var runContext = getArrayAST(`//AssignmentExpression[//Identifier[@name == 'runInNewContext']]`, ast)[0];
    var addImports = Object.values(allImports).map(i => i.id).filter((i, j, arr) => arr.indexOf(i) === j)
    var requireCode = getArrayAST('*', importsTemplate(addImports))[0].body[0];
    requireCode.expression.right = runContext.right;
    runContext.right = requireCode;
}

module.exports = injectImports;
